In [3]:
import pandas as pd
import os
from datetime import datetime
from datetime import datetime, timedelta
import time as time_lib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy.stats import gaussian_kde
import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [4]:
def report_savefig(plotname):
    fig_folder_path = "figures/"
    fig.savefig(fig_folder_path+plotname+".pdf",format='pdf',bbox_inches='tight',pad_inches=0,transparent=True)

# Selected site cloud data (resolution 15 minute)

In [5]:
cloudfolderpath = 'cloud'
filename = 'dataset_non_realistic_cm_r_5x5.csv'
df = pd.read_csv(os.path.join(cloudfolderpath,filename),index_col='Datetime',parse_dates=['Datetime'])
df.head()

,site_name,I,Iclr,latt,long,CI,CI_1,CI_2,CI_3,CI_4,...,R_16,R_17,R_18,R_19,R_20,R_21,R_22,R_23,R_24,R_25
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-04-01 08:00:00+07:00,ISL001,357.6841,402.581345,14.00523,100.519403,14.0,20.0,15.0,15.0,20.0,...,106.0,87.0,82.0,76.0,77.0,94.0,81.0,79.0,75.0,76.0
2022-04-01 08:15:00+07:00,ISL001,413.5059,468.732095,14.00523,100.519403,14.0,20.0,15.0,15.0,20.0,...,106.0,87.0,82.0,76.0,77.0,94.0,81.0,79.0,75.0,76.0
2022-04-01 08:30:00+07:00,ISL001,471.1933,532.843024,14.00523,100.519403,13.0,12.0,15.0,17.0,21.0,...,75.0,74.0,75.0,76.0,78.0,79.0,76.0,76.0,75.0,77.0
2022-04-01 08:45:00+07:00,ISL001,536.3066,594.551229,14.00523,100.519403,13.0,12.0,15.0,17.0,21.0,...,75.0,74.0,75.0,76.0,78.0,79.0,76.0,76.0,75.0,77.0
2022-04-01 09:00:00+07:00,ISL001,604.1268,653.535674,14.00523,100.519403,13.0,12.0,15.0,17.0,21.0,...,75.0,74.0,75.0,76.0,78.0,79.0,76.0,76.0,75.0,77.0


In [6]:
# Site selection
site = "ISL001"

# Read and filter the CSV file in chunks
chunk_size = 10000  # Adjust based on your system's memory capacity
chunks = pd.read_csv(os.path.join(cloudfolderpath,filename), chunksize=chunk_size)

filtered_chunks = []

for chunk in chunks:
    filtered_chunk = chunk[chunk['site_name'] == site]
    filtered_chunks.append(filtered_chunk)

# Concatenate the filtered chunks into a single DataFrame
df = pd.concat(filtered_chunks, ignore_index=True)
# Convert 'Date' column to datetime format
df['Datetime'] = pd.to_datetime(df['Datetime'])
# Set 'Date' as the index
df.set_index('Datetime', inplace=True)
df = df.tz_localize(None)
df.head()

,site_name,I,Iclr,latt,long,CI,CI_1,CI_2,CI_3,CI_4,...,R_16,R_17,R_18,R_19,R_20,R_21,R_22,R_23,R_24,R_25
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-04-01 08:00:00,ISL001,357.6841,402.581345,14.00523,100.519403,14.0,20.0,15.0,15.0,20.0,...,106.0,87.0,82.0,76.0,77.0,94.0,81.0,79.0,75.0,76.0
2022-04-01 08:15:00,ISL001,413.5059,468.732095,14.00523,100.519403,14.0,20.0,15.0,15.0,20.0,...,106.0,87.0,82.0,76.0,77.0,94.0,81.0,79.0,75.0,76.0
2022-04-01 08:30:00,ISL001,471.1933,532.843024,14.00523,100.519403,13.0,12.0,15.0,17.0,21.0,...,75.0,74.0,75.0,76.0,78.0,79.0,76.0,76.0,75.0,77.0
2022-04-01 08:45:00,ISL001,536.3066,594.551229,14.00523,100.519403,13.0,12.0,15.0,17.0,21.0,...,75.0,74.0,75.0,76.0,78.0,79.0,76.0,76.0,75.0,77.0
2022-04-01 09:00:00,ISL001,604.1268,653.535674,14.00523,100.519403,13.0,12.0,15.0,17.0,21.0,...,75.0,74.0,75.0,76.0,78.0,79.0,76.0,76.0,75.0,77.0


In [7]:
df.columns

Index(['site_name', 'I', 'Iclr', 'latt', 'long', 'CI', 'CI_1', 'CI_2', 'CI_3',
       'CI_4', 'CI_5', 'CI_6', 'CI_7', 'CI_8', 'CI_9', 'CI_10', 'CI_11',
       'CI_12', 'CI_14', 'CI_15', 'CI_16', 'CI_17', 'CI_18', 'CI_19', 'CI_20',
       'CI_21', 'CI_22', 'CI_23', 'CI_24', 'CI_25', 'R', 'R_1', 'R_2', 'R_3',
       'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'R_10', 'R_11', 'R_12',
       'R_14', 'R_15', 'R_16', 'R_17', 'R_18', 'R_19', 'R_20', 'R_21', 'R_22',
       'R_23', 'R_24', 'R_25'],
      dtype='object')

# NCEP forecast data

In [8]:
nwp_file_path = "nwp"
filenames = ['site001_2022-01-01to2022-12-31.csv','site001_2023-01-01to2023-07-31.csv']
df_nwp_2022 = pd.read_csv(os.path.join(nwp_file_path,filenames[0]), delimiter=';',parse_dates=[['# Date', 'UT time']])
df_nwp_2023 = pd.read_csv(os.path.join(nwp_file_path,filenames[1]), delimiter=';',parse_dates=[['# Date', 'UT time']])
df_nwp = pd.concat([df_nwp_2022,df_nwp_2023], axis=0)
df_nwp.set_index('# Date_UT time', inplace=True)
df_nwp.index.names = ['Datetime']
df_nwp.index = df_nwp.index.str.replace('24:00', '00:00')
df_nwp.index = pd.to_datetime(df_nwp.index)
df_nwp.head()

,Temperature,Relative Humidity,Pressure,Wind speed,Wind direction,Rainfall,Snowfall,Snow depth,Short-wave irradiation
Datetime,,,,,,,,,
2022-01-01 00:15:00,294.56,66.96,1015.16,3.36,20.53,0.0,0.0,0.0,12.4400
2022-01-01 00:30:00,294.97,65.78,1015.33,3.18,21.46,0.0,0.0,0.0,22.8662
2022-01-01 00:45:00,295.38,64.60,1015.50,3.01,22.51,0.0,0.0,0.0,34.8691
2022-01-01 01:00:00,295.79,63.42,1015.67,2.84,23.68,0.0,0.0,0.0,48.3559
2022-01-01 01:15:00,296.19,62.24,1015.84,2.67,25.01,0.0,0.0,0.0,63.2205


## Unit conversion of forecast data

In [9]:
#Convert Wh/m2 to W/m2
df_nwp['Short-wave irradiation']=df_nwp['Short-wave irradiation']*4
df_nwp['Short-wave irradiation'].describe()

count    55392.000000
mean       227.548257
std        309.122259
min          0.000000
25%          0.000000
50%          4.411400
75%        447.837200
max       1156.288000
Name: Short-wave irradiation, dtype: float64

# Merge cloud and forecast data

In [10]:
# Extract only specific areas from cloud data (no.13 CI ch and R ch)
df_extract = df[['I','Iclr','CI','R']]
# Merge cloud and forecast data
df_all = df_nwp.join(df_extract)
df_all.head()

,Temperature,Relative Humidity,Pressure,Wind speed,Wind direction,Rainfall,Snowfall,Snow depth,Short-wave irradiation,I,Iclr,CI,R
Datetime,,,,,,,,,,,,,
2022-01-01 00:15:00,294.56,66.96,1015.16,3.36,20.53,0.0,0.0,0.0,49.7600,NaN,NaN,NaN,NaN
2022-01-01 00:30:00,294.97,65.78,1015.33,3.18,21.46,0.0,0.0,0.0,91.4648,NaN,NaN,NaN,NaN
2022-01-01 00:45:00,295.38,64.60,1015.50,3.01,22.51,0.0,0.0,0.0,139.4764,NaN,NaN,NaN,NaN
2022-01-01 01:00:00,295.79,63.42,1015.67,2.84,23.68,0.0,0.0,0.0,193.4236,NaN,NaN,NaN,NaN
2022-01-01 01:15:00,296.19,62.24,1015.84,2.67,25.01,0.0,0.0,0.0,252.8820,NaN,NaN,NaN,NaN


In [11]:
# Add clear-sky index
df_all['k_now'] = df_all['I']/df_all['Iclr']

# Normalize cloud index
df_all['CI'] = (df_all['CI']-df_all['CI'].min())/(df_all['CI'].max()-df_all['CI'].min())
df_all['R'] = (df_all['R']-df_all['R'].min())/(df_all['R'].max()-df_all['R'].min())

# Add hour index as a new feature
df_all['hour_index'] = df_all.index.hour-11.5

# Add lag of irradiance as new features
df_all['I_lag30'] = df_all['I'].shift(2)
df_all['I_lag45'] = df_all['I'].shift(3)

# Add lag of cloud index as new features
df_all['CI_lag30'] = df_all['CI'].shift(2)
df_all['R_lag30'] = df_all['R'].shift(2)

df_all.loc['2022-05-01'].head(60)
# df_all.head()

,Temperature,Relative Humidity,Pressure,Wind speed,Wind direction,Rainfall,Snowfall,Snow depth,Short-wave irradiation,I,Iclr,CI,R,k_now,hour_index,I_lag30,I_lag45,CI_lag30,R_lag30
Datetime,,,,,,,,,,,,,,,,,,,
2022-05-01 00:15:00,302.57,69.47,1005.16,1.72,118.55,0.181421,0.0,0.0,208.0228,NaN,NaN,NaN,NaN,NaN,-11.5,NaN,NaN,NaN,NaN
2022-05-01 00:30:00,302.88,67.97,1005.25,1.63,114.13,0.190689,0.0,0.0,261.6488,NaN,NaN,NaN,NaN,NaN,-11.5,NaN,NaN,NaN,NaN
2022-05-01 00:45:00,303.20,66.47,1005.33,1.56,109.25,0.199957,0.0,0.0,317.4732,NaN,NaN,NaN,NaN,NaN,-11.5,NaN,NaN,NaN,NaN
2022-05-01 01:00:00,303.51,64.98,1005.41,1.50,103.94,0.209224,0.0,0.0,375.1952,NaN,NaN,NaN,NaN,NaN,-10.5,NaN,NaN,NaN,NaN
2022-05-01 01:15:00,303.82,63.48,1005.49,1.45,98.23,0.218492,0.0,0.0,434.4940,NaN,NaN,NaN,NaN,NaN,-10.5,NaN,NaN,NaN,NaN
2022-05-01 01:30:00,304.13,61.98,1005.57,1.42,92.20,0.227760,0.0,0.0,495.0284,NaN,NaN,NaN,NaN,NaN,-10.5,NaN,NaN,NaN,NaN
2022-05-01 01:45:00,304.44,60.49,1005.65,1.41,85.97,0.222803,0.0,0.0,550.2908,NaN,NaN,NaN,NaN,NaN,-10.5,NaN,NaN,NaN,NaN
2022-05-01 02:00:00,304.75,58.99,1005.73,1.41,79.67,0.203620,0.0,0.0,598.4312,NaN,NaN,NaN,NaN,NaN,-9.5,NaN,NaN,NaN,NaN
2022-05-01 02:15:00,305.07,57.49,1005.81,1.43,73.46,0.184437,0.0,0.0,644.8496,NaN,NaN,NaN,NaN,NaN,-9.5,NaN,NaN,NaN,NaN


In [12]:
# Drop unnecessary features
df_forecast = df_all.drop(columns=['CI', 'R','k_now','Snowfall','Snow depth'])
df_forecast.rename(columns = {'Relative Humidity':'Relative_Humidity','Wind speed':'Wind_speed','Wind direction':'Wind_direction','Short-wave irradiation':'Short_wave_irradiation'}, inplace = True) 
# Remove NaN data
df_forecast = df_forecast.dropna()
df_forecast.head()

,Temperature,Relative_Humidity,Pressure,Wind_speed,Wind_direction,Rainfall,Short_wave_irradiation,I,Iclr,hour_index,I_lag30,I_lag45,CI_lag30,R_lag30
Datetime,,,,,,,,,,,,,,
2022-04-01 08:45:00,308.13,38.32,1006.13,5.71,28.78,0.0,278.2168,536.3066,594.551229,-3.5,413.5059,357.6841,0.054902,0.305882
2022-04-01 09:00:00,308.06,38.34,1005.97,5.80,28.60,0.0,239.3480,604.1268,653.535674,-2.5,471.1933,413.5059,0.050980,0.290196
2022-04-01 09:15:00,307.70,39.64,1006.15,5.90,30.95,0.0,202.4816,661.4119,709.505601,-2.5,536.3066,471.1933,0.050980,0.290196
2022-04-01 09:30:00,307.33,40.95,1006.33,6.02,33.21,0.0,167.9160,723.4158,762.194726,-2.5,604.1268,536.3066,0.050980,0.290196
2022-04-01 09:45:00,306.97,42.25,1006.51,6.14,35.38,0.0,135.9256,773.8237,811.358317,-2.5,661.4119,604.1268,0.050980,0.290196


In [13]:
# Export the prepared data intended for forecasting
df_forecast.to_csv('site001_dataprepare_forecast.csv')

# Seperate time period for features selection in MATLAB

In [15]:
df_morning = df_forecast.between_time('7:00','9:45')
df_noon = df_forecast.between_time('10:00','15:00')
df_evening = df_forecast.between_time('15:15','17:00')
df_morning.to_csv('features_selection/site001_dataprepare_forecast_morning.csv')
df_noon.to_csv('features_selection/site001_dataprepare_forecast_noon.csv')
df_evening.to_csv('features_selection/site001_dataprepare_forecast_evening.csv')